In [72]:
import pandas as pd
from numpy import log

import plotly.express as px

from skfeature.function.similarity_based import fisher_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
import statistics

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()

,bidder_id,payment_account,address,outcome
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0


In [3]:
bids_df = pd.read_csv('bids.csv')
bids_df.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


In [4]:
bids_df.merchandise.value_counts()

mobile              2126587
jewelry             1902058
sporting goods      1855207
home goods          1224234
office equipment     289838
furniture             99181
computers             81084
books and music       51941
clothing              16447
auto parts             9757
Name: merchandise, dtype: int64

In [5]:
bids_df.dropna(inplace=True)
bids_df.isnull().sum()

bid_id         0
bidder_id      0
auction        0
merchandise    0
device         0
time           0
country        0
ip             0
url            0
dtype: int64

In [12]:
tmp_df = train_df.merge(bids_df, how='left')
bot_ip = tmp_df[tmp_df['outcome'] == 1.0]['ip'].unique()
lst = bot_ip.tolist()
ip_dct = {ip:1 for ip in lst}

In [13]:
def check_ip(ip_list):
    counter = 0
    l = len(ip_list)
    for ip in ip_list:
        if ip in ip_dct:
            counter += 1
    return counter/l

all_bidder_ip = bids_df.groupby('bidder_id')['ip'].unique().reset_index()
all_bidder_ip['prop_bot_ip'] = all_bidder_ip['ip'].apply(check_ip)
all_bidder_ip.head()

,bidder_id,ip,prop_bot_ip
0,001068c415025a009fee375a12cff4fcnht8y,[139.226.147.115],1.000000
1,002d229ffb247009810828f648afc2ef593rb,[37.40.254.131],0.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,[21.67.17.162],1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,"[44.241.8.179, 190.88.89.83, 115.47.140.180]",1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,"[143.118.40.162, 54.212.177.220, 236.63.15.129...",0.600000
...,...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,"[254.230.31.160, 237.38.108.211, 225.146.227.9...",0.087686
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,"[166.44.112.124, 224.130.156.216, 6.162.158.10...",0.722222
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,[67.61.214.36],0.000000
6607,ffd62646d600b759a985d45918bd6f0431vmz,"[201.65.186.165, 93.82.160.29, 63.67.37.44, 22...",1.000000


In [14]:
bids_uniq_df = bids_df.groupby('bidder_id').nunique()
bids_uniq_df.reset_index(inplace=True)
bids_uniq_df['bids_per_url'] = bids_uniq_df['bid_id']/bids_uniq_df['url']
bids_uniq_df['bids_per_auction'] = bids_uniq_df['bid_id']/bids_uniq_df['auction']
bids_uniq_df.head()

,bidder_id,bid_id,auction,merchandise,device,time,country,ip,url,bids_per_url,bids_per_auction
0,001068c415025a009fee375a12cff4fcnht8y,1,1,1,1,1,1,1,1,1.000000,1.000000
1,002d229ffb247009810828f648afc2ef593rb,2,1,1,2,2,1,1,1,2.000000,2.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1,1,1,1,1,1,1,1,1.000000,1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3,3,1,3,3,1,3,2,1.500000,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,20,13,1,8,20,1,10,7,2.857143,1.538462
...,...,...,...,...,...,...,...,...,...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,25075,637,1,792,23487,102,18726,8039,3.119169,39.364207
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22,15,1,13,22,6,18,12,1.833333,1.466667
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1,1,1,1,1,1,1,1,1.000000,1.000000
6607,ffd62646d600b759a985d45918bd6f0431vmz,664,55,1,96,664,1,37,144,4.611111,12.072727


In [30]:
tmp = train_df.merge(bids_uniq_df, how='left')
merged_df = all_bidder_ip.merge(tmp, left_on='bidder_id', right_on='bidder_id')
y = merged_df['outcome']
cols = merged_df[['prop_bot_ip', 'bid_id', 'auction', 'device', 'ip_y', 'url', 'bids_per_url', 'bids_per_auction']]
cols = cols.rename(columns={
    'bid_id':'total_bids',
    'auction':'total_auction',
    'device':'total_device',
    'ip_y':'total_ip',
    'url':'total_url',
})
cols.head()

,prop_bot_ip,total_bids,total_auction,total_device,total_ip,total_url,bids_per_url,bids_per_auction
0,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
1,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
2,0.368932,141.0,74.0,67.0,103.0,84.0,1.678571,1.905405
3,0.666667,3.0,3.0,3.0,3.0,1.0,3.000000,1.000000
4,0.375321,534.0,17.0,164.0,389.0,4.0,133.500000,31.411765


In [60]:
def z_score(df):
    df_std = cols.copy()
    for col in df_std.columns:
        df_std[col]=(df_std[col]-df_std[col].mean())/df_std[col].std()   
    return df_std
    
X_std = z_score(cols)
X_std.head()

,prop_bot_ip,total_bids,total_auction,total_device,total_ip,total_url,bids_per_url,bids_per_auction
0,1.572590,-0.105173,-0.418186,-0.413257,-0.140576,-0.128832,-0.126387,-0.203166
1,1.572590,-0.105173,-0.418186,-0.413257,-0.140576,-0.128832,-0.126387,-0.203166
2,-0.242043,-0.095652,0.077273,-0.061621,-0.119278,-0.097831,-0.122071,-0.173969
3,0.614092,-0.105037,-0.404612,-0.402601,-0.140159,-0.128832,-0.113666,-0.203166
4,-0.223670,-0.068923,-0.309592,0.455178,-0.059558,-0.127712,0.716375,0.777550


In [69]:
all_cols = X_std.join(y)
corr = all_cols.corr()
px.imshow(
    corr,
    template='plotly_dark'
)

In [92]:
ranks = fisher_score.fisher_score(X_std.to_numpy(), y.to_numpy())
feat_importance = pd.Series(ranks, X_std.columns, y)
px.bar(feat_importance, template='plotly_dark')

In [93]:
X = X_std[['prop_bot_ip', 'total_auction', 'total_ip', 'total_url', 'bids_per_url', 'bids_per_auction']]

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=5)

In [95]:
rfc_model = RandomForestClassifier(
    random_state=5,
    n_estimators=800,
    max_depth=15,
    min_samples_split=4,
    min_samples_leaf=3,
    max_features='sqrt',
    max_leaf_nodes=31,
)
rfc = rfc_model.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print(accuracy_score(rfc_pred, y_test))
rfc_pred_proba = rfc.predict_proba(X_test)
rfc_pred_proba = rfc_pred_proba[:,1]
print(roc_auc_score(y_test, rfc_pred_proba))

0.9932773109243698
0.9636810798444291


In [96]:
cbc_model = CatBoostClassifier(
    random_seed=5,
    learning_rate=0.01,
    n_estimators=1000
)
cbc = cbc_model.fit(X_train, y_train, verbose=False)
cbc_pred = cbc.predict(X_test)
print(accuracy_score(cbc_pred, y_test))
cbc_pred_proba = cbc.predict_proba(X_test)
cbc_pred_proba = cbc_pred_proba[:,1]
print(roc_auc_score(y_test, cbc_pred_proba))

0.9915966386554622
0.9885037748798903


In [97]:
gbc_model = GradientBoostingClassifier(
    learning_rate=0.01,
    random_state=5,
    n_estimators=1000,
    min_samples_split=2,
    max_features='sqrt',
    max_leaf_nodes=13
)
gbc = gbc_model.fit(X_train, y_train)
gbc_pred = gbc.predict(X_test)
print(accuracy_score(gbc_pred, y_test))
gbc_pred_proba = gbc.predict_proba(X_test)
gbc_pred_proba = gbc_pred_proba[:,1]
print(roc_auc_score(y_test, gbc_pred_proba))

0.9915966386554622
0.9896190803019904


In [98]:
accuracy_list = []
auc_list = []

kf = KFold(n_splits=10,shuffle=True,random_state=5)

for train_index, test_index in kf.split(X_std):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    estimators=[('rfc', rfc),('cbc',cbc),('gbc',gbc)]
    ensemble = VotingClassifier(estimators, voting='soft')
    ensemble_model = ensemble.fit(X_train, y_train)
    
    y_pred = ensemble_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)
    
    y_pred_proba = rfc_model.predict_proba(X_test)
    y_pred_proba = y_pred_proba[:,1]

    auc_list.append(roc_auc_score(y_test, y_pred_proba))

print('Average accuracy is:', statistics.mean(accuracy_list))
print('Average auc is:', statistics.mean(auc_list))

0:	learn: 0.6650555	total: 2.6ms	remaining: 2.6s
1:	learn: 0.6406911	total: 5.63ms	remaining: 2.81s
2:	learn: 0.6141581	total: 9.49ms	remaining: 3.15s
3:	learn: 0.5911167	total: 12.6ms	remaining: 3.14s
4:	learn: 0.5699559	total: 15.8ms	remaining: 3.15s
5:	learn: 0.5473661	total: 19.4ms	remaining: 3.21s
6:	learn: 0.5254356	total: 22.6ms	remaining: 3.21s
7:	learn: 0.5077722	total: 25.6ms	remaining: 3.17s
8:	learn: 0.4893215	total: 29.1ms	remaining: 3.2s
9:	learn: 0.4721250	total: 32.3ms	remaining: 3.19s
10:	learn: 0.4539960	total: 34.8ms	remaining: 3.13s
11:	learn: 0.4365477	total: 36.8ms	remaining: 3.03s
12:	learn: 0.4203294	total: 38.9ms	remaining: 2.95s
13:	learn: 0.4053985	total: 41ms	remaining: 2.89s
14:	learn: 0.3905537	total: 42.9ms	remaining: 2.81s
15:	learn: 0.3771353	total: 44.7ms	remaining: 2.75s
16:	learn: 0.3630190	total: 47.4ms	remaining: 2.74s
17:	learn: 0.3495079	total: 49.4ms	remaining: 2.69s
18:	learn: 0.3363713	total: 52.1ms	remaining: 2.69s
19:	learn: 0.3245431	total:

211:	learn: 0.0245756	total: 570ms	remaining: 2.12s
212:	learn: 0.0244865	total: 574ms	remaining: 2.12s
213:	learn: 0.0243880	total: 576ms	remaining: 2.12s
214:	learn: 0.0243187	total: 580ms	remaining: 2.12s
215:	learn: 0.0242242	total: 583ms	remaining: 2.12s
216:	learn: 0.0241275	total: 587ms	remaining: 2.12s
217:	learn: 0.0240392	total: 591ms	remaining: 2.12s
218:	learn: 0.0239719	total: 595ms	remaining: 2.12s
219:	learn: 0.0238899	total: 597ms	remaining: 2.12s
220:	learn: 0.0238630	total: 599ms	remaining: 2.11s
221:	learn: 0.0237820	total: 601ms	remaining: 2.11s
222:	learn: 0.0237285	total: 603ms	remaining: 2.1s
223:	learn: 0.0236828	total: 606ms	remaining: 2.1s
224:	learn: 0.0236170	total: 608ms	remaining: 2.09s
225:	learn: 0.0235817	total: 610ms	remaining: 2.09s
226:	learn: 0.0235373	total: 612ms	remaining: 2.08s
227:	learn: 0.0234903	total: 615ms	remaining: 2.08s
228:	learn: 0.0234501	total: 618ms	remaining: 2.08s
229:	learn: 0.0233699	total: 621ms	remaining: 2.08s
230:	learn: 0.

438:	learn: 0.0179822	total: 1.14s	remaining: 1.45s
439:	learn: 0.0179607	total: 1.14s	remaining: 1.45s
440:	learn: 0.0179303	total: 1.14s	remaining: 1.45s
441:	learn: 0.0179212	total: 1.15s	remaining: 1.45s
442:	learn: 0.0179112	total: 1.15s	remaining: 1.44s
443:	learn: 0.0179069	total: 1.15s	remaining: 1.44s
444:	learn: 0.0179018	total: 1.15s	remaining: 1.44s
445:	learn: 0.0178870	total: 1.16s	remaining: 1.44s
446:	learn: 0.0178836	total: 1.16s	remaining: 1.43s
447:	learn: 0.0178714	total: 1.16s	remaining: 1.43s
448:	learn: 0.0178697	total: 1.16s	remaining: 1.43s
449:	learn: 0.0178652	total: 1.17s	remaining: 1.43s
450:	learn: 0.0178638	total: 1.17s	remaining: 1.42s
451:	learn: 0.0178564	total: 1.17s	remaining: 1.42s
452:	learn: 0.0178401	total: 1.17s	remaining: 1.42s
453:	learn: 0.0178015	total: 1.17s	remaining: 1.41s
454:	learn: 0.0177965	total: 1.18s	remaining: 1.41s
455:	learn: 0.0177727	total: 1.18s	remaining: 1.41s
456:	learn: 0.0177407	total: 1.18s	remaining: 1.4s
457:	learn: 0

661:	learn: 0.0152202	total: 1.7s	remaining: 869ms
662:	learn: 0.0152170	total: 1.7s	remaining: 867ms
663:	learn: 0.0151958	total: 1.71s	remaining: 864ms
664:	learn: 0.0151896	total: 1.71s	remaining: 862ms
665:	learn: 0.0151833	total: 1.71s	remaining: 860ms
666:	learn: 0.0151756	total: 1.72s	remaining: 858ms
667:	learn: 0.0151722	total: 1.72s	remaining: 855ms
668:	learn: 0.0151626	total: 1.72s	remaining: 852ms
669:	learn: 0.0151391	total: 1.72s	remaining: 849ms
670:	learn: 0.0151348	total: 1.73s	remaining: 847ms
671:	learn: 0.0151296	total: 1.73s	remaining: 844ms
672:	learn: 0.0151223	total: 1.73s	remaining: 841ms
673:	learn: 0.0151180	total: 1.73s	remaining: 838ms
674:	learn: 0.0151162	total: 1.73s	remaining: 835ms
675:	learn: 0.0151118	total: 1.74s	remaining: 832ms
676:	learn: 0.0150832	total: 1.74s	remaining: 830ms
677:	learn: 0.0150684	total: 1.74s	remaining: 827ms
678:	learn: 0.0150389	total: 1.74s	remaining: 825ms
679:	learn: 0.0150327	total: 1.75s	remaining: 822ms
680:	learn: 0.

888:	learn: 0.0138734	total: 2.27s	remaining: 283ms
889:	learn: 0.0138662	total: 2.27s	remaining: 281ms
890:	learn: 0.0138660	total: 2.27s	remaining: 278ms
891:	learn: 0.0138626	total: 2.28s	remaining: 276ms
892:	learn: 0.0138620	total: 2.28s	remaining: 273ms
893:	learn: 0.0138585	total: 2.28s	remaining: 271ms
894:	learn: 0.0138449	total: 2.29s	remaining: 268ms
895:	learn: 0.0138445	total: 2.29s	remaining: 266ms
896:	learn: 0.0138439	total: 2.29s	remaining: 263ms
897:	learn: 0.0138380	total: 2.29s	remaining: 260ms
898:	learn: 0.0138352	total: 2.29s	remaining: 258ms
899:	learn: 0.0138339	total: 2.3s	remaining: 255ms
900:	learn: 0.0138323	total: 2.3s	remaining: 253ms
901:	learn: 0.0138227	total: 2.3s	remaining: 250ms
902:	learn: 0.0138191	total: 2.3s	remaining: 248ms
903:	learn: 0.0138167	total: 2.31s	remaining: 245ms
904:	learn: 0.0138130	total: 2.31s	remaining: 242ms
905:	learn: 0.0138108	total: 2.31s	remaining: 240ms
906:	learn: 0.0138106	total: 2.31s	remaining: 237ms
907:	learn: 0.01

...

137:	learn: 0.0362406	total: 383ms	remaining: 2.39s
138:	learn: 0.0360507	total: 387ms	remaining: 2.4s
139:	learn: 0.0358095	total: 390ms	remaining: 2.4s
140:	learn: 0.0355512	total: 393ms	remaining: 2.39s
141:	learn: 0.0353403	total: 396ms	remaining: 2.39s
142:	learn: 0.0351698	total: 399ms	remaining: 2.39s
143:	learn: 0.0349283	total: 402ms	remaining: 2.39s
144:	learn: 0.0347214	total: 406ms	remaining: 2.39s
145:	learn: 0.0344722	total: 409ms	remaining: 2.39s
146:	learn: 0.0342573	total: 413ms	remaining: 2.4s
147:	learn: 0.0340795	total: 417ms	remaining: 2.4s
148:	learn: 0.0338478	total: 420ms	remaining: 2.4s
149:	learn: 0.0336084	total: 424ms	remaining: 2.4s
150:	learn: 0.0334307	total: 428ms	remaining: 2.4s
151:	learn: 0.0331896	total: 430ms	remaining: 2.4s
152:	learn: 0.0330084	total: 432ms	remaining: 2.39s
153:	learn: 0.0328603	total: 434ms	remaining: 2.38s
154:	learn: 0.0326864	total: 436ms	remaining: 2.38s
155:	learn: 0.0325285	total: 438ms	remaining: 2.37s
156:	learn: 0.032341

331:	learn: 0.0215798	total: 949ms	remaining: 1.91s
332:	learn: 0.0215711	total: 951ms	remaining: 1.9s
333:	learn: 0.0215339	total: 953ms	remaining: 1.9s
334:	learn: 0.0214935	total: 956ms	remaining: 1.9s
335:	learn: 0.0214501	total: 958ms	remaining: 1.89s
336:	learn: 0.0214282	total: 962ms	remaining: 1.89s
337:	learn: 0.0214107	total: 966ms	remaining: 1.89s
338:	learn: 0.0213893	total: 969ms	remaining: 1.89s
339:	learn: 0.0213647	total: 972ms	remaining: 1.89s
340:	learn: 0.0213274	total: 976ms	remaining: 1.89s
341:	learn: 0.0212814	total: 980ms	remaining: 1.88s
342:	learn: 0.0212713	total: 983ms	remaining: 1.88s
343:	learn: 0.0212526	total: 987ms	remaining: 1.88s
344:	learn: 0.0212200	total: 990ms	remaining: 1.88s
345:	learn: 0.0211899	total: 992ms	remaining: 1.88s
346:	learn: 0.0211286	total: 994ms	remaining: 1.87s
347:	learn: 0.0211031	total: 996ms	remaining: 1.87s
348:	learn: 0.0210858	total: 999ms	remaining: 1.86s
349:	learn: 0.0210412	total: 1s	remaining: 1.86s
350:	learn: 0.0210

528:	learn: 0.0181049	total: 1.52s	remaining: 1.35s
529:	learn: 0.0181031	total: 1.52s	remaining: 1.35s
530:	learn: 0.0180748	total: 1.52s	remaining: 1.34s
531:	learn: 0.0180630	total: 1.53s	remaining: 1.34s
532:	learn: 0.0180549	total: 1.53s	remaining: 1.34s
533:	learn: 0.0180137	total: 1.53s	remaining: 1.34s
534:	learn: 0.0180117	total: 1.54s	remaining: 1.34s
535:	learn: 0.0179750	total: 1.54s	remaining: 1.34s
536:	learn: 0.0179622	total: 1.55s	remaining: 1.33s
537:	learn: 0.0179487	total: 1.55s	remaining: 1.33s
538:	learn: 0.0179378	total: 1.55s	remaining: 1.33s
539:	learn: 0.0179052	total: 1.56s	remaining: 1.32s
540:	learn: 0.0179017	total: 1.56s	remaining: 1.32s
541:	learn: 0.0178909	total: 1.56s	remaining: 1.32s
542:	learn: 0.0178717	total: 1.56s	remaining: 1.31s
543:	learn: 0.0178544	total: 1.56s	remaining: 1.31s
544:	learn: 0.0178416	total: 1.57s	remaining: 1.31s
545:	learn: 0.0178308	total: 1.57s	remaining: 1.3s
546:	learn: 0.0178039	total: 1.57s	remaining: 1.3s
547:	learn: 0.

716:	learn: 0.0162264	total: 2.08s	remaining: 822ms
717:	learn: 0.0162247	total: 2.09s	remaining: 820ms
718:	learn: 0.0162166	total: 2.09s	remaining: 818ms
719:	learn: 0.0161974	total: 2.1s	remaining: 815ms
720:	learn: 0.0161950	total: 2.1s	remaining: 813ms
721:	learn: 0.0161812	total: 2.1s	remaining: 810ms
722:	learn: 0.0161787	total: 2.11s	remaining: 807ms
723:	learn: 0.0161773	total: 2.11s	remaining: 804ms
724:	learn: 0.0161721	total: 2.11s	remaining: 801ms
725:	learn: 0.0161664	total: 2.12s	remaining: 798ms
726:	learn: 0.0161637	total: 2.12s	remaining: 795ms
727:	learn: 0.0161620	total: 2.12s	remaining: 792ms
728:	learn: 0.0161507	total: 2.12s	remaining: 790ms
729:	learn: 0.0161359	total: 2.13s	remaining: 787ms
730:	learn: 0.0161339	total: 2.13s	remaining: 783ms
731:	learn: 0.0161300	total: 2.13s	remaining: 780ms
732:	learn: 0.0161281	total: 2.13s	remaining: 778ms
733:	learn: 0.0161191	total: 2.14s	remaining: 775ms
734:	learn: 0.0161177	total: 2.14s	remaining: 772ms
735:	learn: 0.0

913:	learn: 0.0151394	total: 2.63s	remaining: 248ms
914:	learn: 0.0151258	total: 2.64s	remaining: 245ms
915:	learn: 0.0151194	total: 2.64s	remaining: 242ms
916:	learn: 0.0151151	total: 2.64s	remaining: 239ms
917:	learn: 0.0151069	total: 2.65s	remaining: 237ms
918:	learn: 0.0151021	total: 2.65s	remaining: 234ms
919:	learn: 0.0150874	total: 2.66s	remaining: 231ms
920:	learn: 0.0150816	total: 2.66s	remaining: 228ms
921:	learn: 0.0150810	total: 2.66s	remaining: 225ms
922:	learn: 0.0150805	total: 2.67s	remaining: 222ms
923:	learn: 0.0150795	total: 2.67s	remaining: 220ms
924:	learn: 0.0150786	total: 2.67s	remaining: 217ms
925:	learn: 0.0150780	total: 2.68s	remaining: 214ms
926:	learn: 0.0150721	total: 2.68s	remaining: 211ms
927:	learn: 0.0150687	total: 2.68s	remaining: 208ms
928:	learn: 0.0150675	total: 2.69s	remaining: 205ms
929:	learn: 0.0150659	total: 2.69s	remaining: 203ms
930:	learn: 0.0150653	total: 2.69s	remaining: 200ms
931:	learn: 0.0150640	total: 2.7s	remaining: 197ms
932:	learn: 0